In [4]:
import nibabel as nib
import numpy as np
import surface_distance
from surface_distance import metrics

original_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_splits\original_right.nii.gz"
mirrored_registered_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_splits\mirrored_right.nii.gz"

img_original = nib.load(original_path)
img_mirrored_registered = nib.load(mirrored_registered_path)

mask_original = img_original.get_fdata() > 0
mask_mirrored_registered = img_mirrored_registered.get_fdata() > 0

spacing_mm = img_original.header.get_zooms()[:3]
voxel_volume = spacing_mm[0] * spacing_mm[1] * spacing_mm[2]

V_orig = np.sum(mask_original) * voxel_volume
V_mirr = np.sum(mask_mirrored_registered) * voxel_volume
delta_V = abs(V_orig - V_mirr)
AI_raw = delta_V / ((V_orig + V_mirr) / 2)
AI_percent = AI_raw * 100

surface_distances = surface_distance.compute_surface_distances(
    mask_original, mask_mirrored_registered, spacing_mm
)

asd = surface_distance.compute_average_surface_distance(surface_distances)
hausdorff_100 = surface_distance.compute_robust_hausdorff(surface_distances, 100)
hausdorff_95 = surface_distance.compute_robust_hausdorff(surface_distances, 95)

print(f"Original Volume: {V_orig:.1f} mm³ ({V_orig/1000:.2f} cm³)")
print(f"Mirrored Volume: {V_mirr:.1f} mm³ ({V_mirr/1000:.2f} cm³)")
print(f"Volume Difference: {delta_V:.1f} mm³ ({delta_V/1000:.2f} cm³)")
print(f"Asymmetry Index (0-1): {AI_raw:.4f}")
print(f"Asymmetry Index (%): {AI_percent:.2f}%")
#print(f"Voxel spacing (mm): {tuple(float(v) for v in spacing_mm)}")
print(f"Average Surface Distance (GT→Pred, Pred→GT): ({float(asd[0]):.3f}, {float(asd[1]):.3f}) mm")
print(f"Robust Hausdorff (100%): {float(hausdorff_100):.3f} mm")
print(f"Robust Hausdorff (95%): {float(hausdorff_95):.3f} mm")


Original Volume: 21721.5 mm³ (21.72 cm³)
Mirrored Volume: 22315.6 mm³ (22.32 cm³)
Volume Difference: 594.1 mm³ (0.59 cm³)
Asymmetry Index (0-1): 0.0270
Asymmetry Index (%): 2.70%
Average Surface Distance (GT→Pred, Pred→GT): (1.213, 1.216) mm
Robust Hausdorff (100%): 6.244 mm
Robust Hausdorff (95%): 3.256 mm
